In [8]:
import pandas as pd
import numpy as np
train = pd.read_csv('/Users/imrun/code/machine_learning/ml-study/Housing-Prices-Competition-for-Kaggle-Learn-Users/home-data-for-ml-course/train.csv', index_col = 'Id')
test = pd.read_csv('/Users/imrun/code/machine_learning/ml-study/Housing-Prices-Competition-for-Kaggle-Learn-Users/home-data-for-ml-course/test.csv', index_col = 'Id')
y = np.log1p(train['SalePrice'])
X = train.drop(['SalePrice'], axis=1)
X_test = test.copy()

In [ ]:
X['TotalSF'] = X['TotalBsmtSF'] + X['1stFlrSF'] + X['2ndFlrSF']
X_test['TotalSF'] = X_test['TotalBsmtSF'] + X_test['1stFlrSF'] + X_test['2ndFlrSF']
X['HouseAge'] = X['YrSold'] - X['YearBuilt']
X_test['HouseAge'] = X_test['YrSold'] - X_test['YearBuilt']
X['TotalPorchSF'] = X['OpenPorchSF'] + X['3SsnPorch'] + X['EnclosedPorch'] + X['ScreenPorch']   
X_test['TotalPorchSF'] = X_test['OpenPorchSF'] + X_test['3SsnPorch'] + X_test['EnclosedPorch'] + X_test['ScreenPorch']
X['TotalBath'] = X['FullBath'] + X['HalfBath'] + X['BsmtFullBath'] + X['BsmtHalfBath']
X_test['TotalBath'] = X_test['FullBath'] + X_test['HalfBath'] + X_test['BsmtFullBath'] + X_test['BsmtHalfBath']

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'Wo

In [14]:
!pip3 install lightgbm
!pip3 install catboost
!pip3 install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.8/27.8 MB 8.0 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 5.9 MB/s eta 0:00:00a 0:00:01


In [15]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import Pipeline
from lightgbm import LGBMRegressor
from sklearn.preprocessing import FunctionTransformer

num_cols = X.select_dtypes(include=[np.number]).columns
cat_cols = X.select_dtypes(exclude=[np.number]).columns
num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('log', FunctionTransformer(np.log1p, validate=True))
])
cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', num_pipe, num_cols),
    ('cat', cat_pipe, cat_cols)
])

In [17]:
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
base_learn = [
    ('lgb', LGBMRegressor(learning_rate=0.01, n_estimators=1000, random_state=42, subsample=0.8)),
    ('xgb', XGBRegressor(learning_rate=0.01, n_estimators=1000, random_state=42, subsample=0.8)),
    ('catboost', CatBoostRegressor(learning_rate=0.01, n_estimators=1000, random_state=42, subsample=0.8, verbose=0))
]
stacker = StackingRegressor(
    estimators=base_learn,
    final_estimator=LGBMRegressor(learning_rate=0.01, n_estimators=1000, random_state=42, subsample=0.8),
    cv=KFold(n_splits=5, shuffle=True, random_state=42)
)

In [18]:
model = Pipeline([
    ('preprocessor', preprocessor),
    ('stacker', stacker)
])

model.fit(X, y)
preds = model.predict(X_test)

/opt/homebrew/Caskroom/miniconda/base/envs/myenv/lib/python3.11/site-packages/sklearn/preprocessing/_function_transformer.py:387: RuntimeWarning: invalid value encountered in log1p
  return func(X, **(kw_args if kw_args else {}))


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001038 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3271
[LightGBM] [Info] Number of data points in the train set: 1460, number of used features: 187
[LightGBM] [Info] Start training from score 12.024057
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001639 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3108
[LightGBM] [Info] Number of data points in the train set: 1168, number of used features: 178
[LightGBM] [Info] Start training from score 12.030658


/opt/homebrew/Caskroom/miniconda/base/envs/myenv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000823 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3107
[LightGBM] [Info] Number of data points in the train set: 1168, number of used features: 181
[LightGBM] [Info] Start training from score 12.016898


/opt/homebrew/Caskroom/miniconda/base/envs/myenv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


...

/opt/homebrew/Caskroom/miniconda/base/envs/myenv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000648 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3104
[LightGBM] [Info] Number of data points in the train set: 1168, number of used features: 176
[LightGBM] [Info] Start training from score 12.022040


/opt/homebrew/Caskroom/miniconda/base/envs/myenv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000220 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 1460, number of used features: 3
[LightGBM] [Info] Start training from score 12.024057


/opt/homebrew/Caskroom/miniconda/base/envs/myenv/lib/python3.11/site-packages/sklearn/preprocessing/_function_transformer.py:387: RuntimeWarning: invalid value encountered in log1p
  return func(X, **(kw_args if kw_args else {}))
/opt/homebrew/Caskroom/miniconda/base/envs/myenv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [19]:
submission = pd.DataFrame({
  'Id':       test.index,
  'SalePrice': np.expm1(preds)
})
submission.to_csv('submission.csv', index=False)